In [1]:
import torch
import numpy as np

from utils import Dataset, setup_seed
from train import train
from models import NeuMF, ConvNCF

In [2]:
dataset_path = 'data/lastfm'

# Load the dataset
dataset = Dataset(dataset_path)
train_mat, test_ratings, test_negatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
print('Dataset: #user=%d, #item=%d, #train_pairs=%d, #test_pairs=%d' 
      % (dataset.num_users, dataset.num_items, train_mat.nnz, len(test_ratings)))

mat = train_mat.todense()
mat = 1 * (mat > 0)
if type(mat) is np.matrix: mat = np.asarray(mat)

Dataset: #user=518, #item=3488, #train_pairs=45654, #test_pairs=518


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

users_num, items_num = dataset.num_users, dataset.num_items

### NeuMF

In [3]:
lr = 0.01
weight_decay = 1e-6
embedding_size = 16
num_layers = 1
epochs = 32
batch_size = 1024
mode = 'hr'
topK = 10

hr_list, ndcg_list = [], []
for seed in range(4):
    setup_seed(seed)
    model = NeuMF(users_num, items_num, embedding_size, num_layers)
    model = model.to(device)
    hr, ndcg = train(model, lr, weight_decay, test_ratings, test_negatives, topK, mat, epochs, batch_size, mode, device)
    hr_list.append(hr)
    ndcg_list.append(ndcg)
    
print('lr: ', lr, 'embedding_size: ', embedding_size, 'num_layers: ', num_layers)
print('HR: %.4f' %np.mean(hr_list), 'NDCG: %.4f' %np.mean(ndcg_list))
print('=' * 100)

Init: HR = 0.1081, NDCG = 0.0481
epoch=0, loss=0.4089, HR=0.6236, NDCG=0.4315
epoch=1, loss=0.2865, HR=0.7104, NDCG=0.4912
epoch=2, loss=0.2903, HR=0.7317, NDCG=0.5109
epoch=3, loss=0.2774, HR=0.7452, NDCG=0.5183
epoch=4, loss=0.2432, HR=0.7394, NDCG=0.5303
epoch=5, loss=0.2278, HR=0.7413, NDCG=0.5149
End. Best HR = 0.7452, NDCG = 0.5183. 
----------------------------------------------------------------------------------------------------
Init: HR = 0.1062, NDCG = 0.0485
epoch=0, loss=0.4099, HR=0.5946, NDCG=0.4199
epoch=1, loss=0.3833, HR=0.6988, NDCG=0.4988
epoch=2, loss=0.2993, HR=0.7278, NDCG=0.5185
epoch=3, loss=0.2588, HR=0.7181, NDCG=0.5185
epoch=4, loss=0.2576, HR=0.7413, NDCG=0.5222
epoch=5, loss=0.2026, HR=0.7162, NDCG=0.5134
epoch=6, loss=0.2105, HR=0.7317, NDCG=0.5029
End. Best HR = 0.7413, NDCG = 0.5222. 
----------------------------------------------------------------------------------------------------
Init: HR = 0.0927, NDCG = 0.0389
epoch=0, loss=0.3876, HR=0.6255, NDC

### ConvNCF

In [5]:
lr = 0.01
weight_decay = 1e-6
embedding_size = 64
epochs = 32
batch_size = 1024
mode = 'hr'
topK = 10

hr_list, ndcg_list = [], []
for seed in range(4):
    setup_seed(seed)
    model = ConvNCF(users_num=users_num, items_num=items_num, embedding_size=embedding_size, device=device)
    model = model.to(device)
    hr, ndcg = train(model, lr, weight_decay, test_ratings, test_negatives, topK, mat, epochs, batch_size, mode, device)
    hr_list.append(hr)
    ndcg_list.append(ndcg)
    
print('lr: ', lr, 'embedding_size: ', embedding_size)
print('HR: %.4f' %np.mean(hr_list), 'NDCG: %.4f' %np.mean(ndcg_list))
print('=' * 100)

Init: HR = 0.0907, NDCG = 0.0426
epoch=0, loss=0.4397, HR=0.4788, NDCG=0.3214
epoch=1, loss=0.4433, HR=0.4884, NDCG=0.3245
epoch=2, loss=0.4208, HR=0.5598, NDCG=0.3581
epoch=3, loss=0.3794, HR=0.6120, NDCG=0.3942
epoch=4, loss=0.3591, HR=0.6544, NDCG=0.4407
epoch=5, loss=0.3344, HR=0.6506, NDCG=0.4437
epoch=6, loss=0.2805, HR=0.6429, NDCG=0.4380
End. Best HR = 0.6544, NDCG = 0.4407. 
----------------------------------------------------------------------------------------------------
Init: HR = 0.1236, NDCG = 0.0605
epoch=0, loss=0.5147, HR=0.1834, NDCG=0.0958
epoch=1, loss=0.5232, HR=0.1544, NDCG=0.0936
epoch=2, loss=0.4774, HR=0.0772, NDCG=0.0444
epoch=3, loss=0.5033, HR=0.0000, NDCG=0.0000
End. Best HR = 0.1544, NDCG = 0.0936. 
----------------------------------------------------------------------------------------------------
Init: HR = 0.1293, NDCG = 0.0646
epoch=0, loss=0.4603, HR=0.4788, NDCG=0.3054
epoch=1, loss=0.4522, HR=0.4768, NDCG=0.3422
epoch=2, loss=0.4405, HR=0.4961, NDC